<a href="https://colab.research.google.com/github/Zahramashayekhpour/organic-and-nonorganic-fruit-classification/blob/master/CNNs_as_feature_extractors.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import glob
import cv2
import os
#import mahotas as mt
import seaborn as sns
import pandas as pd
from PIL import Image, ImageStat
import tensorflow as tf
import random
from sklearn.model_selection import train_test_split

In [ ]:
from google.colab import drive   ## allow Colab to connect your Drive
root = '/content/gdrive/'
drive.mount( root )

Drive already mounted at /content/gdrive/; to attempt to forcibly remount, call drive.mount("/content/gdrive/", force_remount=True).


In [ ]:
DATADIR="/content/gdrive/MyDrive/thesis/total-dataset(after segmentation)/thermal/mushroom/total"
CATEGORIES = os.listdir(DATADIR)
print(CATEGORIES)
Organic_DATADIR = "/content/gdrive/MyDrive/thesis/total-dataset(after segmentation)/thermal/mushroom/total/organic"
Non_Organic_DATADIR = "/content/gdrive/MyDrive/thesis/total-dataset(after segmentation)/thermal/mushroom/total/non-organic"


['non-organic', 'organic']


In [ ]:
count = 0
# Iterate directory
for path in os.listdir(Organic_DATADIR):
    # check if current path is a file
    if os.path.isfile(os.path.join(Organic_DATADIR, path)):
        count += 1
print('organic images count:', count)

count = 0
for path in os.listdir(Non_Organic_DATADIR):
    # check if current path is a file
    if os.path.isfile(os.path.join(Non_Organic_DATADIR, path)):
        count += 1
print('non-organic images count:', count)

organic images count: 229
non-organic images count: 227


In [ ]:
training_data = []
IMG_SIZE_H=224 # you need to set up a numerical value here. Useful to resize to normalize data size
IMG_SIZE_W=224 # you need to set up a numerical value here. Useful to resize to normalize data size
               # we define here the size of all dataset for our model
def create_training_data():
    for category in CATEGORIES:  # do plants and weeds

        path = os.path.join(DATADIR,category)  # create path to the labels
                                               ### The os.path.join function accepts a list of paths that you want to merge into one
                                               ### (it seems that,we want to make path consists of dataset and labels )

        class_num = CATEGORIES.index(category)  # get the classification  (0 or a 1). 0=plants 1=weeds
                                                ### go to CATEGORIES and find the index eccording to each category

        for img in os.listdir(path):  # iterate over each image per plants and weeds
                                      ### os.listdir: This method returns the list of all files and directories in the specified path. The return type of this method is list.

          if img.endswith('.jpg'):      ## consider the img format, (just the 'jpg')
            img_array = cv2.imread(os.path.join(path,img))  # convert to array
            new_array = cv2.resize(img_array, (IMG_SIZE_H, IMG_SIZE_W))  # resize to normalize data size
            training_data.append([new_array, class_num])  # The append() method adds an item to the end of the list.add class_num to our training_data



In [ ]:
create_training_data()  # Calling the function for reading images and labels
print(len(training_data)) # Printing the size of the database (number of images in all classes)


453


In [ ]:
random.shuffle(training_data)   ## Shuffle a list (reorganize the order of the list items),
X = []  # An Array for images
y = []  # An Array for labels

#X=training_data[0:-2]
#y=training_data[-1]

for features,label in training_data:   # Seperation of iamegs and labels
    X.append(features)
    y.append(label)
print(np.array(X).shape) # Print the size of the database
print(np.array(y).shape)

(453, 224, 224, 3)
(453,)


In [ ]:
X = np.array(X).reshape(-1, IMG_SIZE_H, IMG_SIZE_W, 3)  # Reshape data in a form that is suitable for keras
y=np.array(y)
print('X:',X.shape) # Print the size of the database
print('y:',y.shape) #

X: (453, 224, 224, 3)
y: (453,)


In [ ]:
import sklearn as sk
from sklearn.neural_network import MLPClassifier
from sklearn import svm
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
import xgboost as xgb
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn import metrics
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold, cross_val_score
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report, confusion_matrix

classifiers = ['LinearDiscriminantAnalysis',
               'LinearSVM', 'RadialSVM',
               'Logistic',  'RandomForest',
               'MLP']

models = [LinearDiscriminantAnalysis(),
          svm.SVC(kernel='linear'),
          svm.SVC(kernel='rbf'),
          LogisticRegression(max_iter = 1000),
          RandomForestClassifier(n_estimators=200, random_state=0),
          MLPClassifier(solver='lbfgs', alpha=1e-5, hidden_layer_sizes=(30, 10), random_state=1)]


def acc_score(df,label):
    Score = pd.DataFrame({"Classifier":classifiers})
    j = 0
    acc = []
    precision=[]
    recall=[]
    #X_train,X_test,Y_train,Y_test = split(df,label)
    for i in models:
        model = i
        model.fit(x_train,y_train)
        predictions = model.predict(x_test)
        acc.append(accuracy_score(y_test,predictions))
        precision.append(precision_score(y_test, predictions))
        recall.append(recall_score(y_test, predictions))

        j = j+1
    Score["Accuracy"] = acc
    Score["precision"]=precision
    Score["recall"]=recall
    #Score.sort_values(by="Accuracy", ascending=False,inplace = True)
    Score.reset_index(drop=True, inplace=True)
    return Score

In [ ]:
# load pretrained MobileNet
model = tf.keras.applications.VGG16(weights='imagenet', include_top=True)

out = model.layers[-2].output    # it is equal to model.layers[-2].output
                                                # Note that model.output gives the last softmax layer
model.new = tf.keras.Model(model.input, out)

X_ml_MobileNet=model.new.predict(X)  #use features from convolutional network for classifier , #This is our X input to classifier
X_ml_MobileNet = X_ml_MobileNet.reshape(X_ml_MobileNet.shape[0], -1)
## normalization
X_ml_MobileNet=(X_ml_MobileNet-X_ml_MobileNet.min())/(X_ml_MobileNet.max()-X_ml_MobileNet.min())
X_ml_MobileNet

X_ml_MobileNet_df=pd.DataFrame(X_ml_MobileNet)

# train is now 75% of the entire data set
x_train, x_test, y_train, y_test = train_test_split(X_ml_MobileNet_df, y, test_size=0.3)

print("Train images:",x_train.shape)
print("Test images:",x_test.shape)
print("Train label:",y_train.shape)
print("Test label:",y_test.shape)




##################
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer

def Log_transform(mat):
    mat = mat - mat.min() + 1
    return np.log(mat)

def fillingEmptyValue(xmat):
    im = SimpleImputer(missing_values=np.nan, strategy="mean", fill_value=None, verbose=0, copy=True)
    xmat = im.fit_transform(xmat)
    return xmat

x_train = fillingEmptyValue(x_train)
x_train = Log_transform(x_train)

x_test = fillingEmptyValue(x_test)
x_test = Log_transform(x_test)

print("x_train : ", x_train.shape ," ,  y_train : ", y_train.shape)
print("x_test : ", x_test.shape ," ,  y_test : ", y_test.shape)

15/15 [==============================] - 2s 110ms/step
Train images: (317, 4096)
Test images: (136, 4096)
Train label: (317,)
Test label: (136,)
x_train :  (317, 4096)  ,  y_train :  (317,)
x_test :  (136, 4096)  ,  y_test :  (136,)


/usr/local/lib/python3.10/dist-packages/sklearn/impute/_base.py:382: FutureWarning: The 'verbose' parameter was deprecated in version 1.1 and will be removed in 1.3. A warning will always be raised upon the removal of empty columns in the future version.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/impute/_base.py:382: FutureWarning: The 'verbose' parameter was deprecated in version 1.1 and will be removed in 1.3. A warning will always be raised upon the removal of empty columns in the future version.
  warnings.warn(


In [ ]:
score_MobileNet = acc_score(x_train,y_train)
score_MobileNet

,Classifier,Accuracy,precision,recall
0,LinearDiscriminantAnalysis,0.970588,0.972973,0.972973
1,LinearSVM,0.992647,0.986667,1.000000
2,RadialSVM,1.000000,1.000000,1.000000
3,Logistic,0.992647,0.986667,1.000000
4,RandomForest,0.985294,1.000000,0.972973
5,MLP,0.970588,0.972973,0.972973


In [ ]:
model = tf.keras.applications.ResNet152(weights='imagenet', include_top=True)

In [ ]:
model.summary()

Model: "resnet152"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_32 (InputLayer)       [(None, 224, 224, 3)]        0         []                            
                                                                                                  
 conv1_pad (ZeroPadding2D)   (None, 230, 230, 3)          0         ['input_32[0][0]']            
                                                                                                  
 conv1_conv (Conv2D)         (None, 112, 112, 64)         9472      ['conv1_pad[0][0]']           
                                                                                                  
 conv1_bn (BatchNormalizati  (None, 112, 112, 64)         256       ['conv1_conv[0][0]']          
 on)                                                                                      

In [ ]:
# load pretrained MobileNet
model = tf.keras.applications.VGG19(weights='imagenet', include_top=True)

out = model.layers[-2].output    # it is equal to model.layers[-2].output
                                                # Note that model.output gives the last softmax layer
model.new = tf.keras.Model(model.input, out)

X_ml_MobileNetV2=model.new.predict(X)  #use features from convolutional network for classifier , #This is our X input to classifier
X_ml_MobileNetV2 = X_ml_MobileNetV2.reshape(X_ml_MobileNetV2.shape[0], -1)
## normalization
X_ml_MobileNetV2=(X_ml_MobileNetV2-X_ml_MobileNetV2.min())/(X_ml_MobileNetV2.max()-X_ml_MobileNetV2.min())
X_ml_MobileNetV2

X_ml_MobileNetV2_df=pd.DataFrame(X_ml_MobileNetV2)

# train is now 75% of the entire data set
x_train, x_test, y_train, y_test = train_test_split(X_ml_MobileNetV2_df, y, test_size=0.3)

print("Train images:",x_train.shape)
print("Test images:",x_test.shape)
print("Train label:",y_train.shape)
print("Test label:",y_test.shape)




##################
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer

def Log_transform(mat):
    mat = mat - mat.min() + 1
    return np.log(mat)

def fillingEmptyValue(xmat):
    im = SimpleImputer(missing_values=np.nan, strategy="mean", fill_value=None, verbose=0, copy=True)
    xmat = im.fit_transform(xmat)
    return xmat

x_train = fillingEmptyValue(x_train)
x_train = Log_transform(x_train)

x_test = fillingEmptyValue(x_test)
x_test = Log_transform(x_test)

print("x_train : ", x_train.shape ," ,  y_train : ", y_train.shape)
print("x_test : ", x_test.shape ," ,  y_test : ", y_test.shape)

15/15 [==============================] - 2s 134ms/step
Train images: (317, 4096)
Test images: (136, 4096)
Train label: (317,)
Test label: (136,)


/usr/local/lib/python3.10/dist-packages/sklearn/impute/_base.py:382: FutureWarning: The 'verbose' parameter was deprecated in version 1.1 and will be removed in 1.3. A warning will always be raised upon the removal of empty columns in the future version.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/impute/_base.py:382: FutureWarning: The 'verbose' parameter was deprecated in version 1.1 and will be removed in 1.3. A warning will always be raised upon the removal of empty columns in the future version.
  warnings.warn(


x_train :  (317, 4096)  ,  y_train :  (317,)
x_test :  (136, 4096)  ,  y_test :  (136,)


In [ ]:
score_VGG19 = acc_score(x_train,y_train)
score_VGG19

,Classifier,Accuracy,precision,recall
0,LinearDiscriminantAnalysis,0.955882,0.961538,0.961538
1,LinearSVM,0.985294,0.975000,1.000000
2,RadialSVM,0.970588,0.962500,0.987179
3,Logistic,0.985294,0.975000,1.000000
4,RandomForest,0.970588,0.962500,0.987179
5,MLP,0.977941,0.962963,1.000000


In [ ]:
# load pretrained MobileNet
model = tf.keras.applications.ResNet101V2(weights='imagenet', include_top=True)

out = model.layers[-2].output    # it is equal to model.layers[-2].output
                                                # Note that model.output gives the last softmax layer
model.new = tf.keras.Model(model.input, out)

X_ml_ResNet101V2=model.new.predict(X)  #use features from convolutional network for classifier , #This is our X input to classifier
X_ml_ResNet101V2 = X_ml_ResNet101V2.reshape(X_ml_ResNet101V2.shape[0], -1)
## normalization
X_ml_ResNet101V2=(X_ml_ResNet101V2-X_ml_ResNet101V2.min())/(X_ml_ResNet101V2.max()-X_ml_ResNet101V2.min())
X_ml_ResNet101V2

X_ml_ResNet101V2_df=pd.DataFrame(X_ml_ResNet101V2)

# train is now 75% of the entire data set
x_train, x_test, y_train, y_test = train_test_split(X_ml_ResNet101V2_df, y, test_size=0.3)

print("Train images:",x_train.shape)
print("Test images:",x_test.shape)
print("Train label:",y_train.shape)
print("Test label:",y_test.shape)




##################
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer

def Log_transform(mat):
    mat = mat - mat.min() + 1
    return np.log(mat)

def fillingEmptyValue(xmat):
    im = SimpleImputer(missing_values=np.nan, strategy="mean", fill_value=None, verbose=0, copy=True)
    xmat = im.fit_transform(xmat)
    return xmat

x_train = fillingEmptyValue(x_train)
x_train = Log_transform(x_train)

x_test = fillingEmptyValue(x_test)
x_test = Log_transform(x_test)

print("x_train : ", x_train.shape ," ,  y_train : ", y_train.shape)
print("x_test : ", x_test.shape ," ,  y_test : ", y_test.shape)

15/15 [==============================] - 4s 142ms/step
Train images: (317, 2048)
Test images: (136, 2048)
Train label: (317,)
Test label: (136,)
x_train :  (317, 2048)  ,  y_train :  (317,)
x_test :  (136, 2048)  ,  y_test :  (136,)


/usr/local/lib/python3.10/dist-packages/sklearn/impute/_base.py:382: FutureWarning: The 'verbose' parameter was deprecated in version 1.1 and will be removed in 1.3. A warning will always be raised upon the removal of empty columns in the future version.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/impute/_base.py:382: FutureWarning: The 'verbose' parameter was deprecated in version 1.1 and will be removed in 1.3. A warning will always be raised upon the removal of empty columns in the future version.
  warnings.warn(


In [ ]:
score_ResNet101V2 = acc_score(x_train,y_train)
score_ResNet101V2

,Classifier,Accuracy,precision,recall
0,LinearDiscriminantAnalysis,0.955882,0.917808,1.000000
1,LinearSVM,0.926471,0.870130,1.000000
2,RadialSVM,0.926471,0.870130,1.000000
3,Logistic,0.897059,0.844156,0.970149
4,RandomForest,0.992647,1.000000,0.985075
5,MLP,0.992647,0.985294,1.000000


In [ ]:
# load pretrained MobileNet
model = tf.keras.applications.ResNet152(weights='imagenet', include_top=True)

out = model.layers[-2].output    # it is equal to model.layers[-2].output
                                                # Note that model.output gives the last softmax layer
model.new = tf.keras.Model(model.input, out)

X_ml_ResNet152=model.new.predict(X)  #use features from convolutional network for classifier , #This is our X input to classifier
X_ml_ResNet152 = X_ml_ResNet152.reshape(X_ml_ResNet152.shape[0], -1)
## normalization
X_ml_ResNet152=(X_ml_ResNet152-X_ml_ResNet152.min())/(X_ml_ResNet152.max()-X_ml_ResNet152.min())
X_ml_ResNet152

X_ml_ResNet152_df=pd.DataFrame(X_ml_ResNet152)

# train is now 75% of the entire data set
x_train, x_test, y_train, y_test = train_test_split(X_ml_ResNet152_df, y, test_size=0.3)

print("Train images:",x_train.shape)
print("Test images:",x_test.shape)
print("Train label:",y_train.shape)
print("Test label:",y_test.shape)




##################
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer

def Log_transform(mat):
    mat = mat - mat.min() + 1
    return np.log(mat)

def fillingEmptyValue(xmat):
    im = SimpleImputer(missing_values=np.nan, strategy="mean", fill_value=None, verbose=0, copy=True)
    xmat = im.fit_transform(xmat)
    return xmat

x_train = fillingEmptyValue(x_train)
x_train = Log_transform(x_train)

x_test = fillingEmptyValue(x_test)
x_test = Log_transform(x_test)

print("x_train : ", x_train.shape ," ,  y_train : ", y_train.shape)
print("x_test : ", x_test.shape ," ,  y_test : ", y_test.shape)

15/15 [==============================] - 5s 205ms/step
Train images: (317, 2048)
Test images: (136, 2048)
Train label: (317,)
Test label: (136,)
x_train :  (317, 2048)  ,  y_train :  (317,)
x_test :  (136, 2048)  ,  y_test :  (136,)


/usr/local/lib/python3.10/dist-packages/sklearn/impute/_base.py:382: FutureWarning: The 'verbose' parameter was deprecated in version 1.1 and will be removed in 1.3. A warning will always be raised upon the removal of empty columns in the future version.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/impute/_base.py:382: FutureWarning: The 'verbose' parameter was deprecated in version 1.1 and will be removed in 1.3. A warning will always be raised upon the removal of empty columns in the future version.
  warnings.warn(


In [ ]:
score_ResNet152 = acc_score(x_train,y_train)
score_ResNet152

,Classifier,Accuracy,precision,recall
0,LinearDiscriminantAnalysis,0.985294,0.973684,1.000000
1,LinearSVM,0.992647,0.986667,1.000000
2,RadialSVM,0.977941,0.961039,1.000000
3,Logistic,0.977941,0.961039,1.000000
4,RandomForest,0.955882,0.959459,0.959459
5,MLP,0.977941,0.973333,0.986486


In [ ]:
# load pretrained MobileNet
model = tf.keras.applications.ResNet101(weights='imagenet', include_top=True)

out = model.layers[-2].output    # it is equal to model.layers[-2].output
                                                # Note that model.output gives the last softmax layer
model.new = tf.keras.Model(model.input, out)

X_ml_ResNet101=model.new.predict(X)  #use features from convolutional network for classifier , #This is our X input to classifier
X_ml_ResNet101 = X_ml_ResNet101.reshape(X_ml_ResNet101.shape[0], -1)
## normalization
X_ml_ResNet101=(X_ml_ResNet101-X_ml_ResNet101.min())/(X_ml_ResNet101.max()-X_ml_ResNet101.min())
X_ml_ResNet101

X_ml_ResNet101_df=pd.DataFrame(X_ml_ResNet101)

# train is now 75% of the entire data set
x_train, x_test, y_train, y_test = train_test_split(X_ml_ResNet101_df, y, test_size=0.3)

print("Train images:",x_train.shape)
print("Test images:",x_test.shape)
print("Train label:",y_train.shape)
print("Test label:",y_test.shape)




##################
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer

def Log_transform(mat):
    mat = mat - mat.min() + 1
    return np.log(mat)

def fillingEmptyValue(xmat):
    im = SimpleImputer(missing_values=np.nan, strategy="mean", fill_value=None, verbose=0, copy=True)
    xmat = im.fit_transform(xmat)
    return xmat

x_train = fillingEmptyValue(x_train)
x_train = Log_transform(x_train)

x_test = fillingEmptyValue(x_test)
x_test = Log_transform(x_test)

print("x_train : ", x_train.shape ," ,  y_train : ", y_train.shape)
print("x_test : ", x_test.shape ," ,  y_test : ", y_test.shape)

15/15 [==============================] - 4s 144ms/step
Train images: (317, 2048)
Test images: (136, 2048)
Train label: (317,)
Test label: (136,)
x_train :  (317, 2048)  ,  y_train :  (317,)
x_test :  (136, 2048)  ,  y_test :  (136,)


/usr/local/lib/python3.10/dist-packages/sklearn/impute/_base.py:382: FutureWarning: The 'verbose' parameter was deprecated in version 1.1 and will be removed in 1.3. A warning will always be raised upon the removal of empty columns in the future version.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/impute/_base.py:382: FutureWarning: The 'verbose' parameter was deprecated in version 1.1 and will be removed in 1.3. A warning will always be raised upon the removal of empty columns in the future version.
  warnings.warn(


In [ ]:
score_ResNet101 = acc_score(x_train,y_train)
score_ResNet101

,Classifier,Accuracy,precision,recall
0,LinearDiscriminantAnalysis,0.985294,0.97561,1.0000
1,LinearSVM,1.000000,1.00000,1.0000
2,RadialSVM,0.992647,1.00000,0.9875
3,Logistic,0.992647,1.00000,0.9875
4,RandomForest,0.992647,1.00000,0.9875
5,MLP,0.992647,1.00000,0.9875


In [ ]:
# load pretrained MobileNet
model = tf.keras.applications.ResNet50V2(weights='imagenet', include_top=True)

out = model.layers[-2].output    # it is equal to model.layers[-2].output
                                                # Note that model.output gives the last softmax layer
model.new = tf.keras.Model(model.input, out)

X_ml_ResNet50V2=model.new.predict(X)  #use features from convolutional network for classifier , #This is our X input to classifier
X_ml_ResNet50V2 = X_ml_ResNet50V2.reshape(X_ml_ResNet50V2.shape[0], -1)
## normalization
X_ml_ResNet50V2=(X_ml_ResNet50V2-X_ml_ResNet50V2.min())/(X_ml_ResNet50V2.max()-X_ml_ResNet50V2.min())
X_ml_ResNet50V2

X_ml_ResNet50V2_df=pd.DataFrame(X_ml_ResNet50V2)

# train is now 75% of the entire data set
x_train, x_test, y_train, y_test = train_test_split(X_ml_ResNet50V2_df, y, test_size=0.3)

print("Train images:",x_train.shape)
print("Test images:",x_test.shape)
print("Train label:",y_train.shape)
print("Test label:",y_test.shape)




##################
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer

def Log_transform(mat):
    mat = mat - mat.min() + 1
    return np.log(mat)

def fillingEmptyValue(xmat):
    im = SimpleImputer(missing_values=np.nan, strategy="mean", fill_value=None, verbose=0, copy=True)
    xmat = im.fit_transform(xmat)
    return xmat

x_train = fillingEmptyValue(x_train)
x_train = Log_transform(x_train)

x_test = fillingEmptyValue(x_test)
x_test = Log_transform(x_test)

print("x_train : ", x_train.shape ," ,  y_train : ", y_train.shape)
print("x_test : ", x_test.shape ," ,  y_test : ", y_test.shape)

15/15 [==============================] - 2s 80ms/step
Train images: (317, 2048)
Test images: (136, 2048)
Train label: (317,)
Test label: (136,)
x_train :  (317, 2048)  ,  y_train :  (317,)
x_test :  (136, 2048)  ,  y_test :  (136,)


/usr/local/lib/python3.10/dist-packages/sklearn/impute/_base.py:382: FutureWarning: The 'verbose' parameter was deprecated in version 1.1 and will be removed in 1.3. A warning will always be raised upon the removal of empty columns in the future version.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/impute/_base.py:382: FutureWarning: The 'verbose' parameter was deprecated in version 1.1 and will be removed in 1.3. A warning will always be raised upon the removal of empty columns in the future version.
  warnings.warn(


In [ ]:
score_ResNet50V2 = acc_score(x_train,y_train)
score_ResNet50V2

,Classifier,Accuracy,precision,recall
0,LinearDiscriminantAnalysis,0.941176,0.969231,0.913043
1,LinearSVM,0.970588,0.957746,0.985507
2,RadialSVM,0.970588,0.957746,0.985507
3,Logistic,0.941176,0.917808,0.971014
4,RandomForest,0.985294,1.000000,0.971014
5,MLP,0.985294,0.985507,0.985507


In [ ]:
# load pretrained MobileNet
model = tf.keras.applications.ResNet50(weights='imagenet', include_top=True)

out = model.layers[-2].output    # it is equal to model.layers[-2].output
                                                # Note that model.output gives the last softmax layer
model.new = tf.keras.Model(model.input, out)

X_ml_ResNet50=model.new.predict(X)  #use features from convolutional network for classifier , #This is our X input to classifier
X_ml_ResNet50 = X_ml_ResNet50.reshape(X_ml_ResNet50.shape[0], -1)
## normalization
X_ml_ResNet50=(X_ml_ResNet50-X_ml_ResNet50.min())/(X_ml_ResNet50.max()-X_ml_ResNet50.min())
X_ml_ResNet50

X_ml_ResNet50_df=pd.DataFrame(X_ml_ResNet50)

# train is now 75% of the entire data set
x_train, x_test, y_train, y_test = train_test_split(X_ml_ResNet50_df, y, test_size=0.3)

print("Train images:",x_train.shape)
print("Test images:",x_test.shape)
print("Train label:",y_train.shape)
print("Test label:",y_test.shape)




##################
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer

def Log_transform(mat):
    mat = mat - mat.min() + 1
    return np.log(mat)

def fillingEmptyValue(xmat):
    im = SimpleImputer(missing_values=np.nan, strategy="mean", fill_value=None, verbose=0, copy=True)
    xmat = im.fit_transform(xmat)
    return xmat

x_train = fillingEmptyValue(x_train)
x_train = Log_transform(x_train)

x_test = fillingEmptyValue(x_test)
x_test = Log_transform(x_test)

print("x_train : ", x_train.shape ," ,  y_train : ", y_train.shape)
print("x_test : ", x_test.shape ," ,  y_test : ", y_test.shape)

15/15 [==============================] - 2s 86ms/step
Train images: (317, 2048)
Test images: (136, 2048)
Train label: (317,)
Test label: (136,)
x_train :  (317, 2048)  ,  y_train :  (317,)
x_test :  (136, 2048)  ,  y_test :  (136,)


/usr/local/lib/python3.10/dist-packages/sklearn/impute/_base.py:382: FutureWarning: The 'verbose' parameter was deprecated in version 1.1 and will be removed in 1.3. A warning will always be raised upon the removal of empty columns in the future version.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/impute/_base.py:382: FutureWarning: The 'verbose' parameter was deprecated in version 1.1 and will be removed in 1.3. A warning will always be raised upon the removal of empty columns in the future version.
  warnings.warn(


In [ ]:
score_ResNet50 = acc_score(x_train,y_train)
score_ResNet50

,Classifier,Accuracy,precision,recall
0,LinearDiscriminantAnalysis,0.955882,0.971014,0.943662
1,LinearSVM,0.985294,0.985915,0.985915
2,RadialSVM,0.963235,0.945946,0.985915
3,Logistic,0.970588,0.958904,0.985915
4,RandomForest,0.955882,0.945205,0.971831
5,MLP,0.970588,0.958904,0.985915


In [ ]:
# load pretrained MobileNet
model = tf.keras.applications.DenseNet201(weights='imagenet', include_top=True)

out = model.layers[-2].output    # it is equal to model.layers[-2].output
                                                # Note that model.output gives the last softmax layer
model.new = tf.keras.Model(model.input, out)

X_ml_DenseNet201=model.new.predict(X)  #use features from convolutional network for classifier , #This is our X input to classifier
X_ml_DenseNet201 = X_ml_DenseNet201.reshape(X_ml_DenseNet201.shape[0], -1)
## normalization
X_ml_DenseNet201=(X_ml_DenseNet201-X_ml_DenseNet201.min())/(X_ml_DenseNet201.max()-X_ml_DenseNet201.min())
X_ml_DenseNet201

X_ml_DenseNet201_df=pd.DataFrame(X_ml_DenseNet201)

# train is now 75% of the entire data set
x_train, x_test, y_train, y_test = train_test_split(X_ml_DenseNet201_df, y, test_size=0.3)

print("Train images:",x_train.shape)
print("Test images:",x_test.shape)
print("Train label:",y_train.shape)
print("Test label:",y_test.shape)




##################
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer

def Log_transform(mat):
    mat = mat - mat.min() + 1
    return np.log(mat)

def fillingEmptyValue(xmat):
    im = SimpleImputer(missing_values=np.nan, strategy="mean", fill_value=None, verbose=0, copy=True)
    xmat = im.fit_transform(xmat)
    return xmat

x_train = fillingEmptyValue(x_train)
x_train = Log_transform(x_train)

x_test = fillingEmptyValue(x_test)
x_test = Log_transform(x_test)

print("x_train : ", x_train.shape ," ,  y_train : ", y_train.shape)
print("x_test : ", x_test.shape ," ,  y_test : ", y_test.shape)

15/15 [==============================] - 5s 130ms/step
Train images: (317, 1920)
Test images: (136, 1920)
Train label: (317,)
Test label: (136,)
x_train :  (317, 1920)  ,  y_train :  (317,)
x_test :  (136, 1920)  ,  y_test :  (136,)


/usr/local/lib/python3.10/dist-packages/sklearn/impute/_base.py:382: FutureWarning: The 'verbose' parameter was deprecated in version 1.1 and will be removed in 1.3. A warning will always be raised upon the removal of empty columns in the future version.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/impute/_base.py:382: FutureWarning: The 'verbose' parameter was deprecated in version 1.1 and will be removed in 1.3. A warning will always be raised upon the removal of empty columns in the future version.
  warnings.warn(


In [ ]:
score_DenseNet201 = acc_score(x_train,y_train)
score_DenseNet201

,Classifier,Accuracy,precision,recall
0,LinearDiscriminantAnalysis,0.985294,0.983871,0.983871
1,LinearSVM,0.985294,0.983871,0.983871
2,RadialSVM,0.985294,0.983871,0.983871
3,Logistic,0.963235,0.938462,0.983871
4,RandomForest,0.985294,0.983871,0.983871
5,MLP,0.985294,0.983871,0.983871


In [ ]:
# load pretrained MobileNet
model = tf.keras.applications.DenseNet169(weights='imagenet', include_top=True)

out = model.layers[-2].output    # it is equal to model.layers[-2].output
                                                # Note that model.output gives the last softmax layer
model.new = tf.keras.Model(model.input, out)

X_ml_DenseNet169=model.new.predict(X)  #use features from convolutional network for classifier , #This is our X input to classifier
X_ml_DenseNet169 = X_ml_DenseNet169.reshape(X_ml_DenseNet169.shape[0], -1)
## normalization
X_ml_DenseNet169=(X_ml_DenseNet169-X_ml_DenseNet169.min())/(X_ml_DenseNet169.max()-X_ml_DenseNet169.min())
X_ml_DenseNet169

X_ml_DenseNet169_df=pd.DataFrame(X_ml_DenseNet169)

# train is now 75% of the entire data set
x_train, x_test, y_train, y_test = train_test_split(X_ml_DenseNet169_df, y, test_size=0.3)

print("Train images:",x_train.shape)
print("Test images:",x_test.shape)
print("Train label:",y_train.shape)
print("Test label:",y_test.shape)




##################
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer

def Log_transform(mat):
    mat = mat - mat.min() + 1
    return np.log(mat)

def fillingEmptyValue(xmat):
    im = SimpleImputer(missing_values=np.nan, strategy="mean", fill_value=None, verbose=0, copy=True)
    xmat = im.fit_transform(xmat)
    return xmat

x_train = fillingEmptyValue(x_train)
x_train = Log_transform(x_train)

x_test = fillingEmptyValue(x_test)
x_test = Log_transform(x_test)

print("x_train : ", x_train.shape ," ,  y_train : ", y_train.shape)
print("x_test : ", x_test.shape ," ,  y_test : ", y_test.shape)

15/15 [==============================] - 4s 101ms/step
Train images: (317, 1664)
Test images: (136, 1664)
Train label: (317,)
Test label: (136,)
x_train :  (317, 1664)  ,  y_train :  (317,)
x_test :  (136, 1664)  ,  y_test :  (136,)


/usr/local/lib/python3.10/dist-packages/sklearn/impute/_base.py:382: FutureWarning: The 'verbose' parameter was deprecated in version 1.1 and will be removed in 1.3. A warning will always be raised upon the removal of empty columns in the future version.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/impute/_base.py:382: FutureWarning: The 'verbose' parameter was deprecated in version 1.1 and will be removed in 1.3. A warning will always be raised upon the removal of empty columns in the future version.
  warnings.warn(


In [ ]:
score_DenseNet169 = acc_score(x_train,y_train)
score_DenseNet169

,Classifier,Accuracy,precision,recall
0,LinearDiscriminantAnalysis,1.000000,1.000000,1.000000
1,LinearSVM,0.985294,0.985075,0.985075
2,RadialSVM,0.985294,0.985075,0.985075
3,Logistic,0.985294,0.971014,1.000000
4,RandomForest,0.992647,1.000000,0.985075
5,MLP,0.985294,0.971014,1.000000


In [ ]:
# load pretrained MobileNet
model = tf.keras.applications.DenseNet121(weights='imagenet', include_top=True)

out = model.layers[-2].output    # it is equal to model.layers[-2].output
                                                # Note that model.output gives the last softmax layer
model.new = tf.keras.Model(model.input, out)

X_ml_DenseNet121=model.new.predict(X)  #use features from convolutional network for classifier , #This is our X input to classifier
X_ml_DenseNet121 = X_ml_DenseNet121.reshape(X_ml_DenseNet121.shape[0], -1)
## normalization
X_ml_DenseNet121=(X_ml_DenseNet121-X_ml_DenseNet121.min())/(X_ml_DenseNet121.max()-X_ml_DenseNet121.min())
X_ml_DenseNet121

X_ml_DenseNet121_df=pd.DataFrame(X_ml_DenseNet121)

# train is now 75% of the entire data set
x_train, x_test, y_train, y_test = train_test_split(X_ml_DenseNet121_df, y, test_size=0.3)

print("Train images:",x_train.shape)
print("Test images:",x_test.shape)
print("Train label:",y_train.shape)
print("Test label:",y_test.shape)




##################
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer

def Log_transform(mat):
    mat = mat - mat.min() + 1
    return np.log(mat)

def fillingEmptyValue(xmat):
    im = SimpleImputer(missing_values=np.nan, strategy="mean", fill_value=None, verbose=0, copy=True)
    xmat = im.fit_transform(xmat)
    return xmat

x_train = fillingEmptyValue(x_train)
x_train = Log_transform(x_train)

x_test = fillingEmptyValue(x_test)
x_test = Log_transform(x_test)

print("x_train : ", x_train.shape ," ,  y_train : ", y_train.shape)
print("x_test : ", x_test.shape ," ,  y_test : ", y_test.shape)

15/15 [==============================] - 4s 83ms/step
Train images: (317, 1024)
Test images: (136, 1024)
Train label: (317,)
Test label: (136,)
x_train :  (317, 1024)  ,  y_train :  (317,)
x_test :  (136, 1024)  ,  y_test :  (136,)


/usr/local/lib/python3.10/dist-packages/sklearn/impute/_base.py:382: FutureWarning: The 'verbose' parameter was deprecated in version 1.1 and will be removed in 1.3. A warning will always be raised upon the removal of empty columns in the future version.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/impute/_base.py:382: FutureWarning: The 'verbose' parameter was deprecated in version 1.1 and will be removed in 1.3. A warning will always be raised upon the removal of empty columns in the future version.
  warnings.warn(


In [ ]:
score_DenseNet121 = acc_score(x_train,y_train)
score_DenseNet121

,Classifier,Accuracy,precision,recall
0,LinearDiscriminantAnalysis,1.000000,1.000000,1.0
1,LinearSVM,1.000000,1.000000,1.0
2,RadialSVM,1.000000,1.000000,1.0
3,Logistic,0.992647,0.985294,1.0
4,RandomForest,1.000000,1.000000,1.0
5,MLP,1.000000,1.000000,1.0


In [ ]:
# load pretrained MobileNet
model = tf.keras.applications.MobileNetV2(weights='imagenet', include_top=True)

out = model.layers[-2].output    # it is equal to model.layers[-2].output
                                                # Note that model.output gives the last softmax layer
model.new = tf.keras.Model(model.input, out)

X_ml_MobileNetV2=model.new.predict(X)  #use features from convolutional network for classifier , #This is our X input to classifier
X_ml_MobileNetV2 = X_ml_MobileNetV2.reshape(X_ml_MobileNetV2.shape[0], -1)
## normalization
X_ml_MobileNetV2=(X_ml_MobileNetV2-X_ml_MobileNetV2.min())/(X_ml_MobileNetV2.max()-X_ml_MobileNetV2.min())
X_ml_MobileNetV2

X_ml_MobileNetV2_df=pd.DataFrame(X_ml_MobileNetV2)

# train is now 75% of the entire data set
x_train, x_test, y_train, y_test = train_test_split(X_ml_MobileNetV2_df, y, test_size=0.3)

print("Train images:",x_train.shape)
print("Test images:",x_test.shape)
print("Train label:",y_train.shape)
print("Test label:",y_test.shape)




##################
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer

def Log_transform(mat):
    mat = mat - mat.min() + 1
    return np.log(mat)

def fillingEmptyValue(xmat):
    im = SimpleImputer(missing_values=np.nan, strategy="mean", fill_value=None, verbose=0, copy=True)
    xmat = im.fit_transform(xmat)
    return xmat

x_train = fillingEmptyValue(x_train)
x_train = Log_transform(x_train)

x_test = fillingEmptyValue(x_test)
x_test = Log_transform(x_test)

print("x_train : ", x_train.shape ," ,  y_train : ", y_train.shape)
print("x_test : ", x_test.shape ," ,  y_test : ", y_test.shape)

15/15 [==============================] - 1s 37ms/step
Train images: (317, 1280)
Test images: (136, 1280)
Train label: (317,)
Test label: (136,)
x_train :  (317, 1280)  ,  y_train :  (317,)
x_test :  (136, 1280)  ,  y_test :  (136,)


/usr/local/lib/python3.10/dist-packages/sklearn/impute/_base.py:382: FutureWarning: The 'verbose' parameter was deprecated in version 1.1 and will be removed in 1.3. A warning will always be raised upon the removal of empty columns in the future version.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/impute/_base.py:382: FutureWarning: The 'verbose' parameter was deprecated in version 1.1 and will be removed in 1.3. A warning will always be raised upon the removal of empty columns in the future version.
  warnings.warn(


In [ ]:
score_MobileNetV2 = acc_score(x_train,y_train)
score_MobileNetV2

,Classifier,Accuracy,precision,recall
0,LinearDiscriminantAnalysis,0.985294,0.968750,1.000000
1,LinearSVM,0.985294,0.983871,0.983871
2,RadialSVM,0.963235,0.967213,0.951613
3,Logistic,0.963235,0.967213,0.951613
4,RandomForest,0.970588,0.967742,0.967742
5,MLP,0.933824,0.964912,0.887097


In [ ]:
X_ml_ResNet152_df = X_ml_ResNet152_df.reset_index(drop=True)
X_ml_DenseNet169_df=X_ml_DenseNet169_df.reset_index(drop=True)
Total_feature=pd.concat([X_ml_ResNet152_df,X_ml_DenseNet169_df],axis=1)
Total_feature

,0,1,2,3,4,5,6,7,8,9,...,1654,1655,1656,1657,1658,1659,1660,1661,1662,1663
0,0.006416,0.000000,0.023091,0.000000,0.0,0.000000,0.0,0.024095,0.010046,0.049468,...,0.246417,0.064907,0.000000,0.740847,0.000000,0.004555,0.001412,0.000185,0.000000,0.050224
1,0.000000,0.000000,0.037324,0.000000,0.0,0.000000,0.0,0.007474,0.030922,0.017830,...,0.224221,0.056139,0.000000,0.466449,0.000208,0.011335,0.002183,0.000000,0.000000,0.025225
2,0.000149,0.006485,0.023548,0.000000,0.0,0.000186,0.0,0.030957,0.082753,0.046975,...,0.322470,0.034849,0.000083,0.438908,0.003452,0.022231,0.017807,0.000137,0.000000,0.042752
3,0.000000,0.003695,0.046272,0.000000,0.0,0.000000,0.0,0.000882,0.006879,0.000000,...,0.161737,0.034089,0.009090,0.232420,0.011383,0.018495,0.024083,0.001016,0.000000,0.054678
4,0.000000,0.003223,0.004322,0.003105,0.0,0.000000,0.0,0.000000,0.007482,0.000000,...,0.183534,0.050703,0.013614,0.268827,0.008960,0.010771,0.042667,0.000112,0.000000,0.032899
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
448,0.000000,0.000000,0.037481,0.003182,0.0,0.000000,0.0,0.000000,0.028806,0.002170,...,0.154409,0.026086,0.003255,0.219846,0.006866,0.007389,0.021157,0.000396,0.000000,0.029349
449,0.000000,0.000000,0.013176,0.000000,0.0,0.000000,0.0,0.007595,0.043871,0.054632,...,0.272380,0.051303,0.000000,0.470617,0.000000,0.015674,0.006990,0.000000,0.000000,0.040174
450,0.000000,0.000000,0.054219,0.000000,0.0,0.000000,0.0,0.008360,0.015483,0.113499,...,0.182585,0.062193,0.000000,0.812804,0.000000,0.000974,0.000764,0.000120,0.000000,0.053004
451,0.000000,0.008400,0.067113,0.000000,0.0,0.000000,0.0,0.000000,0.019237,0.038759,...,0.236178,0.072408,0.000000,0.829914,0.000000,0.002234,0.002753,0.000047,0.000000,0.046493


In [ ]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(Total_feature, y, test_size = 0.30,random_state=2)

In [ ]:
score3 = acc_score(x_train,y_train)
score3

,Classifier,Accuracy,precision,recall
0,LinearDiscriminantAnalysis,1.0,1.0,1.0
1,LinearSVM,1.0,1.0,1.0
2,RadialSVM,1.0,1.0,1.0
3,Logistic,1.0,1.0,1.0
4,RandomForest,1.0,1.0,1.0
5,MLP,1.0,1.0,1.0


In [ ]:
X_ml_ResNet152_df = X_ml_ResNet152_df.reset_index(drop=True)
X_ml_DenseNet169_df=X_ml_DenseNet169_df.reset_index(drop=True)
X_ml_MobileNetV2_df=X_ml_MobileNetV2_df.reset_index(drop=True)
Total_feature2=pd.concat([X_ml_ResNet152_df,X_ml_DenseNet169_df,X_ml_MobileNetV2_df],axis=1)
Total_feature2

,0,1,2,3,4,5,6,7,8,9,...,1270,1271,1272,1273,1274,1275,1276,1277,1278,1279
0,0.006416,0.000000,0.023091,0.000000,0.0,0.000000,0.0,0.024095,0.010046,0.049468,...,0.668420,0.010941,0.037797,0.000000,0.002709,0.000000,0.001147,0.0,0.225174,0.298767
1,0.000000,0.000000,0.037324,0.000000,0.0,0.000000,0.0,0.007474,0.030922,0.017830,...,0.660940,0.000801,0.039291,0.146767,0.004719,0.000000,0.011334,0.0,0.357664,0.507896
2,0.000149,0.006485,0.023548,0.000000,0.0,0.000186,0.0,0.030957,0.082753,0.046975,...,0.637985,0.000000,0.004181,0.000000,0.000000,0.000000,0.011631,0.0,0.175750,0.518499
3,0.000000,0.003695,0.046272,0.000000,0.0,0.000000,0.0,0.000882,0.006879,0.000000,...,0.595048,0.000358,0.020652,0.005999,0.000000,0.001308,0.000000,0.0,0.097767,0.396915
4,0.000000,0.003223,0.004322,0.003105,0.0,0.000000,0.0,0.000000,0.007482,0.000000,...,0.714415,0.000000,0.026208,0.037262,0.000000,0.000000,0.018545,0.0,0.185744,0.358382
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
448,0.000000,0.000000,0.037481,0.003182,0.0,0.000000,0.0,0.000000,0.028806,0.002170,...,0.685266,0.006004,0.088161,0.060617,0.000000,0.000000,0.003818,0.0,0.069824,0.475628
449,0.000000,0.000000,0.013176,0.000000,0.0,0.000000,0.0,0.007595,0.043871,0.054632,...,0.702697,0.000000,0.015611,0.002793,0.000000,0.000000,0.000098,0.0,0.143639,0.401600
450,0.000000,0.000000,0.054219,0.000000,0.0,0.000000,0.0,0.008360,0.015483,0.113499,...,0.638497,0.000000,0.019678,0.000000,0.000199,0.000000,0.023547,0.0,0.317534,0.274488
451,0.000000,0.008400,0.067113,0.000000,0.0,0.000000,0.0,0.000000,0.019237,0.038759,...,0.615739,0.000000,0.003385,0.029233,0.000000,0.000000,0.031099,0.0,0.381694,0.293657


In [ ]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(Total_feature2, y, test_size = 0.30,random_state=2)

In [ ]:
score4 = acc_score(x_train,y_train)
score4

,Classifier,Accuracy,precision,recall
0,LinearDiscriminantAnalysis,0.992647,1.000000,0.985507
1,LinearSVM,1.000000,1.000000,1.000000
2,RadialSVM,0.992647,0.985714,1.000000
3,Logistic,0.992647,0.985714,1.000000
4,RandomForest,1.000000,1.000000,1.000000
5,MLP,0.992647,0.985714,1.000000


In [ ]:
training_data = []
IMG_SIZE_H=299 # you need to set up a numerical value here. Useful to resize to normalize data size
IMG_SIZE_W=299 # you need to set up a numerical value here. Useful to resize to normalize data size
               # we define here the size of all dataset for our model
def create_training_data():
    for category in CATEGORIES:  # do plants and weeds

        path = os.path.join(DATADIR,category)  # create path to the labels
                                               ### The os.path.join function accepts a list of paths that you want to merge into one
                                               ### (it seems that,we want to make path consists of dataset and labels )

        class_num = CATEGORIES.index(category)  # get the classification  (0 or a 1). 0=plants 1=weeds
                                                ### go to CATEGORIES and find the index eccording to each category

        for img in os.listdir(path):  # iterate over each image per plants and weeds
                                      ### os.listdir: This method returns the list of all files and directories in the specified path. The return type of this method is list.

          if img.endswith('.jpg'):      ## consider the img format, (just the 'jpg')
            img_array = cv2.imread(os.path.join(path,img))  # convert to array
            new_array = cv2.resize(img_array, (IMG_SIZE_H, IMG_SIZE_W))  # resize to normalize data size
            training_data.append([new_array, class_num])  # The append() method adds an item to the end of the list.add class_num to our training_data



In [ ]:
create_training_data()  # Calling the function for reading images and labels
print(len(training_data)) # Printing the size of the database (number of images in all classes)


453


In [ ]:
random.shuffle(training_data)   ## Shuffle a list (reorganize the order of the list items),
X = []  # An Array for images
y = []  # An Array for labels

#X=training_data[0:-2]
#y=training_data[-1]

for features,label in training_data:   # Seperation of iamegs and labels
    X.append(features)
    y.append(label)
print(np.array(X).shape) # Print the size of the database
print(np.array(y).shape)

(453, 299, 299, 3)
(453,)


In [ ]:
X = np.array(X).reshape(-1, IMG_SIZE_H, IMG_SIZE_W, 3)  # Reshape data in a form that is suitable for keras
y=np.array(y)
print('X:',X.shape) # Print the size of the database
print('y:',y.shape) #

X: (453, 299, 299, 3)
y: (453,)


In [ ]:
# load pretrained MobileNet
model = tf.keras.applications.Xception(weights='imagenet', include_top=True)

out = model.layers[-2].output    # it is equal to model.layers[-2].output
                                                # Note that model.output gives the last softmax layer
model.new = tf.keras.Model(model.input, out)

X_ml_Xception=model.new.predict(X)  #use features from convolutional network for classifier , #This is our X input to classifier
X_ml_Xception = X_ml_Xception.reshape(X_ml_Xception.shape[0], -1)
## normalization
X_ml_Xception=(X_ml_Xception-X_ml_Xception.min())/(X_ml_Xception.max()-X_ml_Xception.min())
X_ml_Xception

X_ml_Xception_df=pd.DataFrame(X_ml_Xception)

# train is now 75% of the entire data set
x_train, x_test, y_train, y_test = train_test_split(X_ml_Xception_df, y, test_size=0.3)

print("Train images:",x_train.shape)
print("Test images:",x_test.shape)
print("Train label:",y_train.shape)
print("Test label:",y_test.shape)




##################
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer

def Log_transform(mat):
    mat = mat - mat.min() + 1
    return np.log(mat)

def fillingEmptyValue(xmat):
    im = SimpleImputer(missing_values=np.nan, strategy="mean", fill_value=None, verbose=0, copy=True)
    xmat = im.fit_transform(xmat)
    return xmat

x_train = fillingEmptyValue(x_train)
x_train = Log_transform(x_train)

x_test = fillingEmptyValue(x_test)
x_test = Log_transform(x_test)

print("x_train : ", x_train.shape ," ,  y_train : ", y_train.shape)
print("x_test : ", x_test.shape ," ,  y_test : ", y_test.shape)

15/15 [==============================] - 3s 196ms/step
Train images: (317, 2048)
Test images: (136, 2048)
Train label: (317,)
Test label: (136,)
x_train :  (317, 2048)  ,  y_train :  (317,)
x_test :  (136, 2048)  ,  y_test :  (136,)


/usr/local/lib/python3.10/dist-packages/sklearn/impute/_base.py:382: FutureWarning: The 'verbose' parameter was deprecated in version 1.1 and will be removed in 1.3. A warning will always be raised upon the removal of empty columns in the future version.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/impute/_base.py:382: FutureWarning: The 'verbose' parameter was deprecated in version 1.1 and will be removed in 1.3. A warning will always be raised upon the removal of empty columns in the future version.
  warnings.warn(


In [ ]:
score_Xception = acc_score(x_train,y_train)
score_Xception

,Classifier,Accuracy,precision,recall
0,LinearDiscriminantAnalysis,0.904412,0.857143,0.970588
1,LinearSVM,0.970588,0.944444,1.000000
2,RadialSVM,0.970588,0.944444,1.000000
3,Logistic,0.955882,0.930556,0.985294
4,RandomForest,0.992647,1.000000,0.985294
5,MLP,0.985294,0.985294,0.985294


In [ ]:
# load pretrained MobileNet
model = tf.keras.applications.InceptionV3(weights='imagenet', include_top=True)

out = model.layers[-2].output    # it is equal to model.layers[-2].output
                                                # Note that model.output gives the last softmax layer
model.new = tf.keras.Model(model.input, out)

X_ml_InceptionV3=model.new.predict(X)  #use features from convolutional network for classifier , #This is our X input to classifier
X_ml_InceptionV3 = X_ml_InceptionV3.reshape(X_ml_InceptionV3.shape[0], -1)
## normalization
X_ml_InceptionV3=(X_ml_InceptionV3-X_ml_InceptionV3.min())/(X_ml_InceptionV3.max()-X_ml_InceptionV3.min())
X_ml_InceptionV3

X_ml_InceptionV3_df=pd.DataFrame(X_ml_InceptionV3)

# train is now 75% of the entire data set
x_train, x_test, y_train, y_test = train_test_split(X_ml_InceptionV3_df, y, test_size=0.3)

print("Train images:",x_train.shape)
print("Test images:",x_test.shape)
print("Train label:",y_train.shape)
print("Test label:",y_test.shape)




##################
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer

def Log_transform(mat):
    mat = mat - mat.min() + 1
    return np.log(mat)

def fillingEmptyValue(xmat):
    im = SimpleImputer(missing_values=np.nan, strategy="mean", fill_value=None, verbose=0, copy=True)
    xmat = im.fit_transform(xmat)
    return xmat

x_train = fillingEmptyValue(x_train)
x_train = Log_transform(x_train)

x_test = fillingEmptyValue(x_test)
x_test = Log_transform(x_test)

print("x_train : ", x_train.shape ," ,  y_train : ", y_train.shape)
print("x_test : ", x_test.shape ," ,  y_test : ", y_test.shape)

15/15 [==============================] - 3s 106ms/step
Train images: (317, 2048)
Test images: (136, 2048)
Train label: (317,)
Test label: (136,)
x_train :  (317, 2048)  ,  y_train :  (317,)
x_test :  (136, 2048)  ,  y_test :  (136,)


/usr/local/lib/python3.10/dist-packages/sklearn/impute/_base.py:382: FutureWarning: The 'verbose' parameter was deprecated in version 1.1 and will be removed in 1.3. A warning will always be raised upon the removal of empty columns in the future version.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/impute/_base.py:382: FutureWarning: The 'verbose' parameter was deprecated in version 1.1 and will be removed in 1.3. A warning will always be raised upon the removal of empty columns in the future version.
  warnings.warn(


In [ ]:
score_InceptionV3= acc_score(x_train,y_train)
score_InceptionV3

,Classifier,Accuracy,precision,recall
0,LinearDiscriminantAnalysis,0.955882,0.924051,1.000000
1,LinearSVM,0.955882,0.935065,0.986301
2,RadialSVM,0.955882,0.935065,0.986301
3,Logistic,0.919118,0.878049,0.986301
4,RandomForest,0.977941,0.986111,0.972603
5,MLP,0.970588,0.960000,0.986301


In [ ]:
# load pretrained MobileNet
model = tf.keras.applications.InceptionResNetV2(weights='imagenet', include_top=True)

out = model.layers[-2].output    # it is equal to model.layers[-2].output
                                                # Note that model.output gives the last softmax layer
model.new = tf.keras.Model(model.input, out)

X_ml_InceptionResNetV2=model.new.predict(X)  #use features from convolutional network for classifier , #This is our X input to classifier
X_ml_InceptionResNetV2 = X_ml_InceptionResNetV2.reshape(X_ml_InceptionResNetV2.shape[0], -1)
## normalization
X_ml_InceptionResNetV2=(X_ml_InceptionResNetV2-X_ml_InceptionResNetV2.min())/(X_ml_InceptionResNetV2.max()-X_ml_InceptionResNetV2.min())
X_ml_InceptionResNetV2

X_ml_InceptionResNetV2_df=pd.DataFrame(X_ml_InceptionResNetV2)

# train is now 75% of the entire data set
x_train, x_test, y_train, y_test = train_test_split(X_ml_InceptionResNetV2_df, y, test_size=0.3)

print("Train images:",x_train.shape)
print("Test images:",x_test.shape)
print("Train label:",y_train.shape)
print("Test label:",y_test.shape)




##################
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer

def Log_transform(mat):
    mat = mat - mat.min() + 1
    return np.log(mat)

def fillingEmptyValue(xmat):
    im = SimpleImputer(missing_values=np.nan, strategy="mean", fill_value=None, verbose=0, copy=True)
    xmat = im.fit_transform(xmat)
    return xmat

x_train = fillingEmptyValue(x_train)
x_train = Log_transform(x_train)

x_test = fillingEmptyValue(x_test)
x_test = Log_transform(x_test)

print("x_train : ", x_train.shape ," ,  y_train : ", y_train.shape)
print("x_test : ", x_test.shape ," ,  y_test : ", y_test.shape)

15/15 [==============================] - 7s 268ms/step
Train images: (317, 1536)
Test images: (136, 1536)
Train label: (317,)
Test label: (136,)
x_train :  (317, 1536)  ,  y_train :  (317,)
x_test :  (136, 1536)  ,  y_test :  (136,)


/usr/local/lib/python3.10/dist-packages/sklearn/impute/_base.py:382: FutureWarning: The 'verbose' parameter was deprecated in version 1.1 and will be removed in 1.3. A warning will always be raised upon the removal of empty columns in the future version.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/impute/_base.py:382: FutureWarning: The 'verbose' parameter was deprecated in version 1.1 and will be removed in 1.3. A warning will always be raised upon the removal of empty columns in the future version.
  warnings.warn(


In [ ]:
score_InceptionResNetV2= acc_score(x_train,y_train)
score_InceptionResNetV2

/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:541: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


,Classifier,Accuracy,precision,recall
0,LinearDiscriminantAnalysis,0.933824,0.900000,0.986301
1,LinearSVM,0.897059,0.904110,0.904110
2,RadialSVM,0.852941,0.863014,0.863014
3,Logistic,0.860294,0.885714,0.849315
4,RandomForest,0.926471,0.956522,0.904110
5,MLP,0.970588,0.972603,0.972603
